In [1]:
import numpy as np
import cvxpy as cvx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


def linear_kernel(x1, x2):
    
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    
    return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2)))


class SVM(object):
    
    def __init__(self, kernel='linear', C=1.0):
        self.kernel = kernel
        self.C = C
    
        if self.C is not None:
            self.C = float(self.C)

    def _kernel_transform(self, X_train, kernel):
        if kernel == 'linear':
            kfunc = linear_kernel
        elif kernel == 'polynomial':
            kfunc = polynomial_kernel
        elif kernel == 'gaussian':
            kfunc = gaussian_kernel
            
        self.kfunc = kfunc
        
        n_samples, n_features = X_train.shape
        self.n_samples = n_samples 
        self.n_features = n_features 
    
        K = np.zeros((n_samples, n_samples))
        
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = kfunc(X_train[i], X_train[j])
             
        return K

    def __solve(self, K_matrix, y_train, C):
        
        n = y_train.shape[0] 
        y_= np.diag(y_train)  # 180 by 180(1 & -1) 
        alpha = cvx.Variable(n) 
        # constraints 
        constraints = []
        for i in range(n):
            constraints += [alpha[i] >= 0, alpha[i] <= C] 
        constraints += [y_train * alpha == 0]
        self.model_obj = cvx.Maximize(np.ones(n) * alpha - .5 * cvx.quad_form(alpha, y_.dot(K_matrix).dot(y_)))#convex식 
        #quad form 
        self.model = cvx.Problem(objective=self.model_obj, constraints=constraints)
        self.model.solve() 
        
        
        #support vector 
        self.a = np.array(alpha.value).flatten()
        self.sv = self.a > 1e-5 
        self.ind = np.arange(len(self.a))[self.sv] 
        self.a = self.a[self.sv] 
        self.sv_y = y_train[self.sv]
        self.sv_x= self.X_train[self.sv] 
        
        # intercept
        self.b = 0 
        for n in range(len(self.a)):
            self.b+=self.sv_y[n]
            self.b-=np.sum(self.a*self.sv_y*K_matrix[self.ind[n],self.sv])
        self.b/=len(self.a)
        
        #weight vector
        if self.kernel == 'linear':
            self.w = np.zeros(self.n_features)
            for n in range(len(self.a)):
                    self.w += self.a[n] * self.sv_y[n] * self.sv_x[n]
        else:
            self.w = None
            
    
    def predict(self,X):
        if self.w is not None:
           
            return np.dot(X,self.w) + self.b
          
        else: 
            y_predict = np.zeros(len(X))#
            
            for i in range(len(X)):
                s=0
                for a, sv_y, sv_x in zip(self.a, self.sv_y, self.sv_x):
                    s += a * sv_y * self.kfunc(X[i], sv_x)
                y_predict[i] = s
                
            return y_predict + self.b 
    def signpredict(self, X_test):
        
        return np.sign(self.predict(X_test))
    
    def fit(self, X_train, y_train):
        self.X_train = X_train 
        K = self._kernel_transform(X_train, self.kernel) 
        self.K = K
        self.__solve(K, y_train, self.C)


In [2]:
def linear_kernel(x1, x2):
    
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    
    return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2)))


In [3]:
def gen_non_lin_separable_data():
    mean1 = [-1,2]
    mean2 = [1,-1]
    mean3 = [4,-4]
    mean4 = [-4,4]
    cov = [[1.0,0.8],[0.8,1.0]]

    X1 = np.random.multivariate_normal(mean1, cov, 50) 
    X1 = np.vstack((X1, np.random.multivariate_normal(mean3,cov,50)))
    y1 = np.ones(len(X1)) 
    X2 = np.random.multivariate_normal(mean2, cov, 50)
    X2 = np.vstack((X2,np.random.multivariate_normal(mean4,cov,50)))
    y2 = np.ones(len(X2))*-1

    return X1, y1, X2, y2

In [4]:
def split_train(X1,y1,X2,y2): 
    scaler = StandardScaler()

    X1_train = X1[:90]
    y1_train = y1[:90]
    X2_train = X2[:90]
    y2_train = y2[:90]
    X_train = np.vstack((X1_train, X2_train))
    y_train = np.hstack((y1_train, y2_train))

    X1_test = X1[90:]
    y1_test = y1[90:]
    X2_test = X2[90:]
    y2_test = y2[90:]
    X_test = np.vstack((X1_test,X2_test))
    y_test = np.hstack((y1_test,y2_test))

    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, y_train, X_test, y_test 

In [5]:
X1,y1,X2,y2 = gen_non_lin_separable_data()
X_train, y_train, X_test, y_test = split_train(X1,y1,X2,y2)
print(X_train, y_train, X_test, y_test)

[[-0.08416118  0.87424993]
 [-0.29413337  0.49728174]
 [ 0.20184886  1.22644509]
 [-0.44840861  0.65768047]
 [-0.43460045  0.21661382]
 [-0.70647151  0.53850861]
 [-0.31037416  0.65628005]
 [-0.0866202   0.70597422]
 [ 0.24870362  0.97990057]
 [-0.56737416  0.25198728]
 [-0.4844301   0.46171077]
 [-0.52159438  0.56676433]
 [-0.53331827  0.42117356]
 [-0.75376584  0.3859173 ]
 [-0.21122294  0.61452898]
 [-0.3550963   0.27682721]
 [-0.07205677  0.73713468]
 [-0.43382132  0.46181742]
 [-0.58629832  0.54017719]
 [-0.89042671  0.26614826]
 [-0.73095358  0.27920298]
 [-0.52612959  0.57619213]
 [-0.00584922  0.84687844]
 [-0.34545065  0.42971592]
 [-1.33727595 -0.22950536]
 [-0.44075478  0.23271837]
 [-0.70247702 -0.03574038]
 [ 0.03433897  1.13253349]
 [-0.43856028  0.52753603]
 [-0.63934571  0.75516959]
 [-0.65874154  0.27474184]
 [-0.52629907  0.54215817]
 [-0.88299928  0.22367018]
 [-0.2147967   0.72191437]
 [ 0.07933281  0.88340343]
 [-0.68012065  0.40744464]
 [-0.47250237  0.16121657]
 

In [6]:
import generate as g
import svm_model as s

X1,y1,X2,y2 = g.gen_non_lin_separable_data()#
X_train, y_train, X_test, y_test = g.split_train(X1,y1,X2,y2)
print(X_train, y_train, X_test, y_test)
print("X_train:",len(X_train),"X_test:",len(X_test),"Y_train:",len(y_train),"Y_test:",len(y_test))

[[-4.27313176e-01  2.94128308e-01]
 [-3.69869669e-01  5.11486491e-01]
 [-3.26202385e-01  7.06333688e-01]
 [-5.34509214e-02  7.75114771e-01]
 [-2.05011528e-01  7.20955929e-01]
 [-6.64677957e-01  3.40063834e-01]
 [-3.71672632e-01  3.71354938e-01]
 [ 1.09217374e-01  7.05001394e-01]
 [-3.73639958e-01  4.23231278e-01]
 [-4.23830775e-01  4.26944479e-01]
 [-2.19093984e-01  5.52986179e-01]
 [-1.85699978e-01  6.85938635e-01]
 [ 1.30289030e-02  9.03538278e-01]
 [-6.88624293e-01  3.84464449e-01]
 [-2.72067392e-01  8.73809950e-01]
 [ 3.49541485e-01  1.19529789e+00]
 [-8.39879153e-01  7.30041370e-01]
 [-1.12792827e-01  8.81583197e-01]
 [-3.40222696e-01  2.18999867e-01]
 [-5.91651632e-01  3.45184323e-01]
 [-3.26446024e-01  3.27120597e-01]
 [ 1.37694386e-01  9.40245311e-01]
 [-9.03950584e-01  4.20323684e-01]
 [-7.65589000e-02  7.03536825e-01]
 [-4.48917700e-01  1.03593144e-01]
 [-7.84460088e-01  3.27498849e-01]
 [-4.21418557e-01  5.47521289e-01]
 [-2.74974065e-01  7.04844466e-01]
 [-6.69116976e-01  6

In [7]:
self= SVM(kernel = 'gaussian', C=100.00)

In [8]:
self.fit(X_train, y_train)

In [9]:
def fit(self, X_train, y_train):
        self.X_train = X_train 
        K = self._kernel_transform(X_train, self.kernel) 
        self.K = K
        self.__solve(K, y_train, self.C)

In [10]:
def _kernel_transform(self, X_train, kernel):
        #choose Kernel 
        if kernel == 'linear':
            kfunc = linear_kernel
        elif kernel == 'polynomial':
            kfunc = polynomial_kernel
        elif kernel == 'gaussian':
            kfunc = gaussian_kernel
            
        self.kfunc = kfunc
        
        # Kernel Matrix
        #Number of observations and features in model(input:X_train)
        n_samples, n_features = X_train.shape
        self.n_samples = n_samples 
        self.n_features = n_features 
        
        K = np.zeros((n_samples, n_samples))
        
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = kfunc(X_train[i], X_train[j]) #(180*180)
             
        return K

In [11]:
# use cvxpy to solve dual equation (convex optimization problem)
# cvxpy modeling: features, constraints, optimal equation
#               : alpha, w, b, support vectors(x & y)
def __solve(self, K_matrix, y_train, C):
      
    n = y_train.shape[0]  
    y_= np.diag(y_train) # 180 by 180(composed of 1 & -1)
    
    alpha = cvx.Variable(n)  

In [12]:
n = y_train.shape[0]  
alpha = cvx.Variable(n)  
C = self.C
y_= np.diag(y_train) 

In [13]:
# alpha constraints 
constraints = []
for i in range(n):
    constraints += [alpha[i] >= 0, alpha[i] <= C] 
constraints += [y_train * alpha == 0]

In [14]:
from IPython.display import Math
Math(r's.t. \sum_{i=1}^na_iy_i = 0 \\and\\ 0<=a_i<=C')

<IPython.core.display.Math object>

In [15]:
# model's optimal equation(dual function) and result 
self.model_obj = cvx.Maximize(np.ones(n) * alpha - .5 * cvx.quad_form(alpha, y_.dot(self.K).dot(y_))) 
self.model = cvx.Problem(objective=self.model_obj, constraints=constraints)
self.model.solve() 

10648.289541109767

In [16]:
from IPython.display import Math
Math(r'max L_D = \sum_{i=1}^na_i - \frac{1}{2}\sum_{i=1}^n\sum_{j=1}^na_ia_jy_iy_jK(x_i,x_j)')

<IPython.core.display.Math object>

In [17]:
 # alphas, support vectors 
# 2-dimension -> 1-dimension
self.a = np.array(alpha.value).flatten()
# constraint: a > 0
self.sv = self.a > 1e-5  
self.ind = np.arange(len(self.a))[self.sv] 
self.a = self.a[self.sv] 
self.sv_y = y_train[self.sv]
self.sv_x= self.X_train[self.sv]  

In [18]:
# alphas
print('- alphas of the model:\n', self.a)

- alphas of the model:
 [9.99955132e+01 9.99988993e+01 1.00001964e+02 1.00003905e+02
 1.00002648e+02 9.99946341e+01 9.99969076e+01 1.00003371e+02
 9.99976215e+01 9.99973969e+01 1.00000198e+02 1.00002213e+02
 1.00005879e+02 9.99951393e+01 1.00004698e+02 1.01667145e-02
 9.99997667e+01 1.00005277e+02 9.99950399e+01 9.99951958e+01
 9.99965579e+01 1.00006598e+02 9.99941973e+01 1.00002834e+02
 9.99928240e+01 9.99936048e+01 9.99991624e+01 1.00002148e+02
 1.00000056e+02 9.99962004e+01 9.99959217e+01 9.45720928e-03
 9.99947517e+01 1.00005390e+02 1.00006162e+02 9.99898775e+01
 1.00005715e+02 1.00001860e+02 9.99962042e+01 1.00006980e+02
 9.99986296e+01 9.99984565e+01 1.00008081e+02 8.86850616e-03
 1.00000943e+02 1.00003543e+02 1.00004992e+02 1.00003394e+02
 7.78392619e-03 5.01649865e+01 9.99953242e+01 9.99932488e+01
 9.99951729e+01 9.99898539e+01 9.99950063e+01 7.14233420e+01
 9.99959361e+01 9.99947260e+01 9.99931100e+01 9.99928448e+01
 9.99922363e+01 9.99945778e+01 9.99943282e+01 9.99935706e+01


In [19]:
# support vector's x values
print('- support vector x values of the model:\n',self.sv_x)
len(self.sv_x)

- support vector x values of the model:
 [[-4.27313176e-01  2.94128308e-01]
 [-3.69869669e-01  5.11486491e-01]
 [-3.26202385e-01  7.06333688e-01]
 [-5.34509214e-02  7.75114771e-01]
 [-2.05011528e-01  7.20955929e-01]
 [-6.64677957e-01  3.40063834e-01]
 [-3.71672632e-01  3.71354938e-01]
 [ 1.09217374e-01  7.05001394e-01]
 [-3.73639958e-01  4.23231278e-01]
 [-4.23830775e-01  4.26944479e-01]
 [-2.19093984e-01  5.52986179e-01]
 [-1.85699978e-01  6.85938635e-01]
 [ 1.30289030e-02  9.03538278e-01]
 [-6.88624293e-01  3.84464449e-01]
 [-2.72067392e-01  8.73809950e-01]
 [ 3.49541485e-01  1.19529789e+00]
 [-8.39879153e-01  7.30041370e-01]
 [-1.12792827e-01  8.81583197e-01]
 [-3.40222696e-01  2.18999867e-01]
 [-5.91651632e-01  3.45184323e-01]
 [-3.26446024e-01  3.27120597e-01]
 [ 1.37694386e-01  9.40245311e-01]
 [-9.03950584e-01  4.20323684e-01]
 [-7.65589000e-02  7.03536825e-01]
 [-4.48917700e-01  1.03593144e-01]
 [-7.84460088e-01  3.27498849e-01]
 [-4.21418557e-01  5.47521289e-01]
 [-2.74974065e

145

In [20]:
# support vector's y values
print('- support vector y values of the model:\n',self.sv_y)
len(self.sv_y)

- support vector y values of the model:
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1.]


145

In [21]:
# intercept of the model
self.b = 0 
for n in range(len(self.a)):
    self.b+=self.sv_y[n]
    self.b-=np.sum(self.a*self.sv_y*self.K[self.ind[n],self.sv])
self.b/=len(self.a)

In [22]:
from IPython.display import Math
Math(r'b = y_i-\sum_{i=1}^n(a_iy_iK(x_i,x_j))')


<IPython.core.display.Math object>

In [23]:
# intercept of the model
print('- intercept of the model:\n',self.b)

- intercept of the model:
 2.9617596594418565


In [24]:
# weight vector of the model
if self.kernel == 'linear':
    self.w = np.zeros(self.n_features)
    for n in range(len(self.a)):
            self.w += self.a[n] * self.sv_y[n] * self.sv_x[n]
else:
    self.w = None

In [25]:
from IPython.display import Math
Math(r' w = \sum_{i=1}^n(a_iy_ix_i)')

<IPython.core.display.Math object>

In [26]:
# weight vector of the model 
print('- Weight vectors of the model:\n',self.w)

- Weight vectors of the model:
 None


In [27]:
#predicted y from model
def predict(self,X):  
        if self.w is not None:
                   # wxT+b
            return np.dot(X,self.w) + self.b 
          
        else: 
            y_predict = np.zeros(len(X))   
            
            for i in range(len(X)):
                s=0
                for a, sv_y, sv_x in zip(self.a, self.sv_y, self.sv_x):
                    s += a * sv_y * self.kfunc(X[i], sv_x)  
                y_predict[i] = s
                   # w*kernel(x)+b
            return y_predict + self.b  

In [28]:
from IPython.display import Math
Math(r' w = \sum_{i=1}^n(a_iy_iK(x_i,x_j))')

<IPython.core.display.Math object>

In [29]:
# predicted y values of X_test
print('- Predicted y values of X_test:\n',self.predict(X_test))#( 754 * 2)


- Predicted y values of X_test:
 [ 1.07958037  1.25146727  0.84618494  1.43626338  1.17584217  0.99564685
  0.68038154  1.28146651  0.8954287   0.74844478  0.16329767 -0.56797206
 -0.97897578 -1.83883499 -0.80229008 -0.52702835 -0.3263386  -0.83638139
  0.07598409 -0.82155574]


In [30]:
def signpredict(self, X_test):
                # signs of y_test
        return np.sign(self.predict(X_test))
    

In [31]:
from IPython.display import Math
Math(r'f(x) = sign(y_i-\sum_{i=1}^n(a_iy_iK(x_i,x_j))')

<IPython.core.display.Math object>

In [32]:
# predicted signs of y values of X_test 
y_predict = self.signpredict(X_test) 
print('- Predicted signs of y values of X_test:\n',y_predict)

- Predicted signs of y values of X_test:
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1.
  1. -1.]


In [33]:
correct = np.sum(y_predict == y_test)
print("%d out of %d predictions correct" % (correct, len(y_predict)))

18 out of 20 predictions correct
